In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
datagen=ImageDataGenerator(
rescale=1.0/255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
fill_mode="nearest",
horizontal_flip=True,

)

In [4]:
df_train="/kaggle/input/jellyfish-types/Train_Test_Valid/Train"

In [5]:
train_data=datagen.flow_from_directory(
    df_train,
    target_size=(256,256),
    batch_size=32,
    class_mode="categorical"
)

Found 900 images belonging to 6 classes.


In [6]:
df_test="/kaggle/input/jellyfish-types/Train_Test_Valid/test"
test_data=datagen.flow_from_directory(
    df_test,
    target_size=(256,256),
    batch_size=32,
    class_mode="categorical"
)

Found 40 images belonging to 6 classes.


In [7]:
img,lab=next(train_data)

# # Deep learning

In [8]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,GlobalMaxPooling2D,Input,Dropout,BatchNormalization
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [28]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
restore_best_weights=True,
verbose=1
)

In [16]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(64,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)
F=Flatten()(M)
D=Dense(32,activation="relu")(F)
outputs=Dense(6,activation="softmax")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1048576)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │    33,554,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,574,438 (128.08 MB)

 Trainable params: 33,574,246 (128.08 MB)

 Non-trainable params: 192 (768.00 B)

In [17]:
model.fit(train_data,epochs=50,validation_data=test_data,callbacks=[early_stopping])

Epoch 1/50


2024-09-28 12:20:55.753029: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,32,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,256,256]{3,2,1,0}, f32[32,64,256,256]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 12:20:56.297787: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.544900699s
Trying algorithm eng0{} for conv (f32[64,32,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,256,256]{3,2,1,0}, f32[32,64,256,256]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_con

29/29 ━━━━━━━━━━━━━━━━━━━━ 33s 638ms/step - accuracy: 0.1870 - loss: 290.2718 - val_accuracy: 0.1000 - val_loss: 173.0825
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 460ms/step - accuracy: 0.1612 - loss: 3.7211 - val_accuracy: 0.0500 - val_loss: 742.2603
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 471ms/step - accuracy: 0.1629 - loss: 3.6665 - val_accuracy: 0.1750 - val_loss: 1389.7833
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 465ms/step - accuracy: 0.1690 - loss: 3.6410 - val_accuracy: 0.1750 - val_loss: 1874.3864
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 460ms/step - accuracy: 0.1733 - loss: 3.6176 - val_accuracy: 0.1750 - val_loss: 2145.0630
Epoch 5: early stopping


In [18]:
loss,accuracy=model.evaluate(test_data)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.1688 - loss: 2197.5330


# # m2

In [19]:
inputs2=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs2)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(64,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(128,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(256,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)
F=Flatten()(M)
D=Dense(32,activation="relu")(F)
outputs2=Dense(6,activation="softmax")(D)
model2=Model(inputs2,outputs2)
model2.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
model2.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1048576)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │    33,554,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,944,998 (129.49 MB)

 Trainable params: 33,944,038 (129.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [20]:
model2.fit(train_data,epochs=50,validation_data=test_data,callbacks=[early_stopping])

Epoch 1/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 46s 836ms/step - accuracy: 0.1636 - loss: 51.1868 - val_accuracy: 0.2000 - val_loss: 66.3199
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 493ms/step - accuracy: 0.1784 - loss: 10.8080 - val_accuracy: 0.1750 - val_loss: 1065.2858
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 469ms/step - accuracy: 0.1709 - loss: 9.9268 - val_accuracy: 0.1500 - val_loss: 2313.6953
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 492ms/step - accuracy: 0.1747 - loss: 8.0634 - val_accuracy: 0.1500 - val_loss: 3238.4299
Epoch 5: early stopping


In [21]:
loss,accuracy=model2.evaluate(test_data)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.1417 - loss: 3333.5813


# # m3

In [23]:
inputs3=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs3)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(64,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(128,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(256,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(556,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(1056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)
F=Flatten()(M)
D=Dense(32,activation="relu")(F)
outputs3=Dense(6,activation="softmax")(D)
model3=Model(inputs3,outputs3)
model3.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
model3.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 64, 64, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 64, 64, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 64, 64, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 64, 64, 1056)   │     5,285,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 64, 64, 1056)   │         4,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 64, 64, 1056)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 1056)   │             

 Total params: 41,566,882 (158.57 MB)

 Trainable params: 41,562,698 (158.55 MB)

 Non-trainable params: 4,184 (16.34 KB)

In [24]:
model3.fit(train_data,epochs=50,validation_data=test_data,callbacks=[early_stopping])

Epoch 1/50


2024-09-28 12:27:50.564998: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=4,k3=0} for conv (f32[32,1056,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,556,64,64]{3,2,1,0}, f32[1056,556,3,3]{3,2,1,0}, f32[1056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 12:27:50.746850: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.18196566s
Trying algorithm eng11{k2=4,k3=0} for conv (f32[32,1056,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,556,64,64]{3,2,1,0}, f32[1056,556,3,3]{3,2,1,0}, f32[1056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={

14/29 ━━━━━━━━━━━━━━━━━━━━ 8s 548ms/step - accuracy: 0.1409 - loss: 528.5502

2024-09-28 12:28:58.844170: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[1056,556,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,556,64,64]{3,2,1,0}, f32[4,1056,64,64]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 12:29:01.152102: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 3.308047963s
Trying algorithm eng0{} for conv (f32[1056,556,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,556,64,64]{3,2,1,0}, f32[4,1056,64,64]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_con

29/29 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.1496 - loss: 379.3502 - val_accuracy: 0.3500 - val_loss: 47.3395
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 547ms/step - accuracy: 0.1572 - loss: 44.3044 - val_accuracy: 0.2000 - val_loss: 390.3674
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 534ms/step - accuracy: 0.1840 - loss: 41.2785 - val_accuracy: 0.1250 - val_loss: 998.9443
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 565ms/step - accuracy: 0.1684 - loss: 38.0358 - val_accuracy: 0.1250 - val_loss: 1496.2162
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 538ms/step - accuracy: 0.1773 - loss: 34.3018 - val_accuracy: 0.1250 - val_loss: 1605.7305
Epoch 5: early stopping


In [27]:
loss,accuracy=model3.evaluate(test_data)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.1354 - loss: 1614.7533


# # m4

In [29]:
inputs4=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs4)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(64,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(128,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(256,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(556,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(1056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(2056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
C=Conv2D(4056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)
F=Flatten()(M)
D=Dense(32,activation="relu")(F)
outputs4=Dense(6,activation="softmax")(D)
model4=Model(inputs4,outputs4)
model4.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
model4.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 64, 64, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 64, 64, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 64, 64, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 64, 64, 1056)   │     5,285,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 64, 64, 1056)   │         4,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 64, 64, 1056)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 32, 32, 1056)   │             

 Total params: 134,813,634 (514.27 MB)

 Trainable params: 134,797,226 (514.21 MB)

 Non-trainable params: 16,408 (64.09 KB)

In [30]:
model4.fit(train_data,epochs=50,validation_data=test_data,callbacks=[early_stopping])

Epoch 1/50


2024-09-28 12:37:19.808547: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=4,k3=0} for conv (f32[32,2056,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1056,32,32]{3,2,1,0}, f32[2056,1056,3,3]{3,2,1,0}, f32[2056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 12:37:19.910415: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.101969355s
Trying algorithm eng11{k2=4,k3=0} for conv (f32[32,2056,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1056,32,32]{3,2,1,0}, f32[2056,1056,3,3]{3,2,1,0}, f32[2056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

18/29 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.1755 - loss: 503.2261

2024-09-28 12:41:28.197167: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[2056,1056,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,1056,32,32]{3,2,1,0}, f32[4,2056,32,32]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 12:41:30.283606: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 3.086548857s
Trying algorithm eng0{} for conv (f32[2056,1056,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,1056,32,32]{3,2,1,0}, f32[4,2056,32,32]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1819 - loss: 456.2538

2024-09-28 12:42:15.808335: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=4,k3=0} for conv (f32[8,4056,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,2056,32,32]{3,2,1,0}, f32[4056,2056,3,3]{3,2,1,0}, f32[4056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 12:42:15.853642: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.045500376s
Trying algorithm eng11{k2=4,k3=0} for conv (f32[8,4056,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,2056,32,32]{3,2,1,0}, f32[4056,2056,3,3]{3,2,1,0}, f32[4056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config=

29/29 ━━━━━━━━━━━━━━━━━━━━ 331s 3s/step - accuracy: 0.1821 - loss: 452.3374 - val_accuracy: 0.1500 - val_loss: 161.0514
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.1773 - loss: 149.5033 - val_accuracy: 0.1500 - val_loss: 114.7066
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.1613 - loss: 105.4643 - val_accuracy: 0.1500 - val_loss: 81.5176
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.1721 - loss: 75.4870 - val_accuracy: 0.1500 - val_loss: 61.2843
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.1798 - loss: 58.7274 - val_accuracy: 0.2000 - val_loss: 6774.6235
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.1773 - loss: 49.0681 - val_accuracy: 0.2000 - val_loss: 6864.4126
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.1441 - loss: 40.5405 - val_accuracy: 0.2000 - val_loss: 6641.3047
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.1706 - loss: 34.6911 - val_accuracy: 0.

In [31]:
loss,accuracy=model4.evaluate(test_data)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.1313 - loss: 61.2844


# # m5

In [45]:
inputs5=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs5)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(M)

C=Conv2D(64,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(128,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(D)

C=Conv2D(256,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(556,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(D)

C=Conv2D(1056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2))(D)

C=Conv2D(2056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(D)

C=Conv2D(4056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(D)

C=Conv2D(8056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(D)

C=Conv2D(1256,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(D)

C=Conv2D(2456,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
B=BatchNormalization()(C)
D=Dropout(0.5)(B)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(D)

# C=Conv2D(5056,(3,3),activation='relu',padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
# B=BatchNormalization()(C)
# D=Dropout(0.5)(B)
# M=MaxPooling2D(pool_size=(2,2))(D)
F=Flatten()(M)
D=Dense(32,activation="relu")(F)
outputs5=Dense(6,activation="softmax")(D)
model5=Model(inputs5,outputs5)
model5.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
model5.summary()

ResourceExhaustedError: {{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[3,3,2056,4056] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2]

In [ ]:
model5.fit(train_data,epochs=50,validation_data=test_data,callbacks=[early_stopping])

In [ ]:
loss,accuracy=model5.evaluate(test_data)

In [13]:
!pip install rembg

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 86.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


In [14]:
# from rembg import remove
# from PIL import Image

In [15]:
# new_data=[]
# for img,label in train_data:
# #     data=Image.open(img)
#     data_rem_background=remove(img)
#     new_data.append(data_rem_background)

TypeError: Cannot handle this data type: (1, 1, 256, 3), <f4